In [3]:
!pip install langchain-community openai pymysql sqlalchemy faiss-cpu pandas tiktoken

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 889.5 kB/s eta 0:00:00a 0:00:01
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)


**Thư viện**

In [1]:
import os
import pandas as pd
from datetime import datetime

from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatAnthropic
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.agents import create_react_agent
from langchain_core.messages import HumanMessage
from sqlalchemy import create_engine
import numpy as np
import json

In [3]:
class QueryAgent:
    def __init__(self):

        # --- 1. Load Database ---
        engine = create_engine(
            f"mysql+pymysql://{os.getenv('MYSQL_USER')}:{os.getenv('MYSQL_PASSWORD')}@"
            f"{os.getenv('MYSQL_HOST')}:{os.getenv('MYSQL_PORT')}/{os.getenv('MYSQL_DB')}"
        )
        # data được lưu trong file .env
        # đọc dữ liệu từ MySQL database
        self.df = pd.read_sql("SELECT * FROM salein_thuc_xuat", con=engine)  # chỉnh lại table

        # --- 2. Init Embedding & LLM ---
        # Khởi tạo mô hình ngôn ngữ
        self.llm = ChatAnthropic(model=os.getenv('CLAUDE_3_5_SONNET'), temperature=0)
        # Embed các trường thông tin thành Vector
        self.embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

        # --- 3. Create vector for each column ---
        self.columns_to_embed = ['name', 'industry', 'description']
        self.embedded_columns = {}
        for col in self.columns_to_embed:
            texts = self.df[col].fillna("").astype(str).tolist()
            vectors = self.embedding_model.embed_documents(texts)
            self.embedded_columns[col] = np.array(vectors)

        # --- 4. Save entire FAISS index (as fallback or hybrid) ---
        self.full_texts = self.df[self.columns_to_embed].astype(str).agg(" ".join, axis=1).tolist()
        self.vectorstore = FAISS.from_texts(self.full_texts, embedding=self.embedding_model)

    # Đánh trọng số cho các trường thông tin
    '''
    Đây là bước quan trọng bởi mô hình LLM sẽ dựa vào trọng số để ưu tiên thực hiện Similiarity Search trong Vector DB
    '''
    def _get_weights_from_llm(self, query: str):
        prompt = PromptTemplate.from_template("""
Bạn là trợ lý phân tích truy vấn người dùng. Dưới đây là truy vấn:

"{query}"

Dựa vào truy vấn trên, hãy gán trọng số (từ 0 đến 1, tổng tối đa là 1) cho các thuộc tính sau:

- name
- industry
- description

Trả lời bằng JSON như sau:
{{"name": 0.3, "industry": 0.4, "description": 0.3}}
        """)
        message = HumanMessage(content=prompt.format(query=query))
        result = self.llm.invoke([message])
        try:
            weights = json.loads(result.content.strip())
            return weights
        except Exception as e:
            print("[Warning] LLM weight parsing failed, using default:", e)
            return {"name": 0.3, "industry": 0.4, "description": 0.3}

    def _compute_query_vector(self, query, weights):
        # Lấy embedding riêng cho từng thuộc tính
        vec = np.zeros_like(next(iter(self.embedded_columns.values()))[0])
        for col in self.columns_to_embed:
            sub_vec = self.embedding_model.embed_query(query)
            vec += weights.get(col, 0) * np.array(sub_vec)
        return vec

    def _search_top_k(self, query_vec, k=5):
        # Kết hợp từ các embedding riêng
        all_vectors = np.stack([self.embedded_columns[col] for col in self.columns_to_embed], axis=0)
        weights = np.array([1.0] * len(self.columns_to_embed))[:, None, None]
        merged_vecs = np.average(all_vectors, axis=0, weights=weights.squeeze())
        similarities = np.dot(merged_vecs, query_vec) / (
            np.linalg.norm(merged_vecs, axis=1) * np.linalg.norm(query_vec)
        )
        top_indices = similarities.argsort()[::-1][:k]
        return self.df.iloc[top_indices]

    def query(self, question: str) -> str:
        # --- 1. Get weights from LLM ---
        weights = self._get_weights_from_llm(question)

        # --- 2. Build query vector ---
        query_vec = self._compute_query_vector(question, weights)

        # --- 3. Search ---
        result_df = self._search_top_k(query_vec)

        # --- 4. Summarize or use ReAct Agent ---
        content = result_df.head(3).to_string(index=False)

        system_prompt = f"""
Bạn là chuyên gia tài chính. Đây là dữ liệu phù hợp với truy vấn "{question}":
{content}

Hãy viết câu trả lời cho người dùng, bằng tiếng Việt, ngắn gọn và đầy đủ. Định dạng số liệu với đơn vị nếu cần.
        """
        response = self.llm.invoke([HumanMessage(content=system_prompt)])
        return response.content

def main():
    agent = QueryAgent()
    questions = [
        "Top 3 công ty công nghệ có tăng trưởng cao nhất",
        "Ngành nào có ROE cao nhất 2023?",
        "FPT làm ăn thế nào 5 năm gần đây?"
    ]
    for q in questions:
        print(f"\n❓ Question: {q}")
        print("📌 Answer:", agent.query(q))

if __name__ == "__main__":
    main()

/var/folders/zg/_wfh70cn569_5p6d24bj1_tr0000gn/T/ipykernel_11543/2369654993.py:15: LangChainDeprecationWarning: The class `ChatAnthropic` was deprecated in LangChain 0.0.28 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-anthropic package and should be used instead. To use it run `pip install -U :class:`~langchain-anthropic` and import as `from :class:`~langchain_anthropic import ChatAnthropic``.
  self.llm = ChatAnthropic(model=os.getenv('CLAUDE_3_5_SONNET'), temperature=0)


ValidationError: 1 validation error for ChatAnthropic
  Value error, Did not find anthropic_api_key, please add an environment variable `ANTHROPIC_API_KEY` which contains it, or pass `anthropic_api_key` as a named parameter. [type=value_error, input_value={'model': 'claude-2', 'te...sable_streaming': False}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error